# CryptoPunks Exploratory Case Study

## Introduction

An NFT is a Non Fungible Token, unique tokens that can be traded and exchanged on the Ethereum blockchain.

CryptoPunks is a collection of 10,000 unique collectible characters, each is a 24x24 pixel, 8-bit-style unique avatar, with proof of ownership stored on the Ethereum blockchain. Their creation began as an experiment, conducted by software developers Matt Hall and John Watkinson in 2017. The Cryptopunks were the inspiration for the ERC-721 standard that powers most digital art and collectibles today. NOTE: Cryptopunks are NOT strictly ERC-20 or ERC-721 tokens (closer to a "custom" ERC-20).

CryptoPunks is the very first NFT series to catch popular interest and one of the most actively traded today. As of April 2022, the market capitalization of all 10,000 CryptoPunks is estimated at around 1,543,250.40 ETH, or over $5B USD (Source: https://nftvaluations.com/cryptopunks).

### Imports

In [7]:
import pandas as pd
import numpy as np
import random
import time

import os
import requests
import json
from dotenv import load_dotenv

import plotly.express as px
import matplotlib.pyplot as plt
import hvplot
import seaborn as sns
%matplotlib inline
import PIL
from PIL import Image
import glob
import cv2 as cv
from IPython import display


from pathlib import Path
from tqdm import tqdm

## Part 1: Data Collection
### Loading Up and Cleaning the Data

In [167]:
# Data for the most up to date transaction information and the full history of calls to the CryptoPunks contract by date. (Sourced from Etherscan.io)

transactions_df = pd.read_csv('data/transactions.csv', index_col = 'DateTime', parse_dates = True, infer_datetime_format = True, low_memory=False)
transactions_df = transactions_df[['Txhash', 'Blockno', 'From','To', 'Value_IN(ETH)', 'TxnFee(ETH)', 'TxnFee(USD)', 'Historical $Price/Eth', 'Method']]
columns = ['txn_hash', 'block', 'from', 'to', 'value_eth', 'txn_fee_eth', 'txn_fee_usd', 'price', 'contract_call']
transactions_df.columns = columns

# Data for transaction history with Cryptopunk attributes. (Sourced from Kaggle) 

tx_attributes_df = pd.read_json("data/txn_history-2021-10-07.jsonl", lines=True)
tx_attributes_df['date'] = pd.to_datetime(tx_attributes_df.date)
tx_attributes_df = tx_attributes_df[["txn_type", "date", "eth", "punk_id", "type", "accessories"]]
tx_attributes_df = tx_attributes_df.explode("type")
tx_attributes_df['num_attributes'] = tx_attributes_df.accessories.apply(lambda x: len(x))

# Data for Artwork Characteristics
punks = pd.read_csv('data/attributes.csv', index_col = 'id')

# Building transaction data with punk IDs
punks_transfers = pd.DataFrame(tx_attributes_df[tx_attributes_df.txn_type == 'Transfer'])[['date', 'punk_id']].set_index(['punk_id', 'date'])

# Building sale data with punk IDs
punks_sales = pd.DataFrame(tx_attributes_df[tx_attributes_df.txn_type == 'Sold'])[['date', 'punk_id', 'eth']].set_index(['punk_id', 'date'])

### Previewing the Data

In [92]:
'''

Transactions DataFrame with the most up to date list of contract calls.

--------------------------------------------------------------------------------------------------
Columns:
-DateTime [Index]: Time at which the transaction was processed [in UTC] => DateTime
-txn_hash: Transaction hash. Can be used to identify specific transactions on a block explorer. => String
-block: Block number where the transaction was recorded. => Integer
-from: Address for the sender of the transaction. => String
-to: Address that received the transaction [CryptoPunks contract]. => String
-value_eth: Value transferred into the contract. => Float
-txn_fee_eth: Gas fee in Ether. => Float
-txn_fee_usd: Gas fee converted to USD (based on price of Ether at the time of the transaction). => Float
-price: Price of Ether in USD at the time of the call. => Float
-contract_call: Type of function that was called (from the contract) for each transaction. => String
--------------------------------------------------------------------------------------------------

'''
transactions_df.tail()

,txn_hash,block,from,to,value_eth,txn_fee_eth,txn_fee_usd,price,contract_call
DateTime,,,,,,,,,
2022-04-09 12:49:46,0x12b731b6c657b3db035493f4a51f7b777488d22bfd33...,14551591,0xe83c750b2708320bb134796c555b80df39a3d97b,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,0.0,0.001706,5.488097,NaN,Offer Punk For Sale
2022-04-09 12:50:35,0x61d36019e596db7820f8d116a4ebd5d676068961d8d3...,14551595,0xe83c750b2708320bb134796c555b80df39a3d97b,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,0.0,0.001706,5.488097,NaN,Offer Punk For Sale
2022-04-09 12:50:35,0xf3736cbcd73b72086fdac45c3b5a2e9ef0c895bad331...,14551595,0xe83c750b2708320bb134796c555b80df39a3d97b,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,0.0,0.001706,5.488097,NaN,Offer Punk For Sale
2022-04-09 14:38:00,0x2f70728819df0616a95cec87838c1aada4a4f6bc505f...,14552090,0x0a9cf96cbe9ce142723caa5c2b6a54bf22a1ebc5,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,0.0,0.001913,6.154825,NaN,Transfer Punk
2022-04-09 15:18:34,0x81c06cd81857bdc12d22fd43f127c4f9b3533b6f57eb...,14552276,0x8bebdb7f971d126bed0084c4cc651be40bee0519,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,0.0,0.003823,12.299534,NaN,Offer Punk For Sale


In [93]:
'''

Transaction history with information for the specific Punk that was involved.

--------------------------------------------------------------------------------------------------
Columns:
-txn_type: Type of function that was called (from the contract) for each transaction. => String
-date: Date for each transaction (only up to the day). => DateTime
-eth: Value of the transaction. => Float
-punk_id: Unique ID for each CryptoPunk. => Integer
-type: Type of CryptoPunk (Female, Male, Alien, Ape). => String
-accessories: List of accessories belonging to the specific Punk. => List of Strings
-num_attributes: How many attributes does the specific CryptoPunk have. => Integer
--------------------------------------------------------------------------------------------------

'''
tx_attributes_df.head()

,txn_type,date,eth,punk_id,type,accessories,num_attributes
0,Bid Withdrawn,2021-09-04,321.0,0,Female,"[Green Eye Shadow, Earring, Blonde Bob]",3
1,Bid,2021-09-01,321.0,0,Female,"[Green Eye Shadow, Earring, Blonde Bob]",3
2,Bid Withdrawn,2021-09-01,320.0,0,Female,"[Green Eye Shadow, Earring, Blonde Bob]",3
3,Bid,2021-09-01,320.0,0,Female,"[Green Eye Shadow, Earring, Blonde Bob]",3
4,Bid Withdrawn,2021-09-01,263.0,0,Female,"[Green Eye Shadow, Earring, Blonde Bob]",3


In [94]:
'''

Artwork Characteristics DataFrame

--------------------------------------------------------------------------------------------------
Columns:
-id [Index]: Unique identifier for each CryptoPunk
-type: Species of each CryptoPunk. Possible options are Human, Alien, Ape, or Zombie.
-gender: Gender of each CryptoPunk. Possible options are Male or Female
-skin_tone: Skin tone of each *Human* CryptoPunk. Non-human punks have Null values. Possible skin tones are Light, Medium, Dark, or Albino.
-count: Number of accessories of each CryptoPunk. 
-accessories: String containing the accessories of each CryptoPunk.
--------------------------------------------------------------------------------------------------

'''
punks.head()

,type,gender,skin_tone,count,accessories
id,,,,,
0,Human,Female,Medium,3,Green Eye Shadow / Earring / Blonde Bob
1,Human,Male,Dark,2,Smile / Mohawk
2,Human,Female,Light,1,Wild Hair
3,Human,Male,Dark,3,Wild Hair / Nerd Glasses / Pipe
4,Human,Male,Medium,4,Big Shades / Wild Hair / Earring / Goat


In [115]:
'''
Transfer History DataFrame

Groups the data by Punk ID and aggregates the amount of times it's been transferred.

IMPORTANT NOTE: Times Transferred != Times Sold. Times Transferred keeps track of how many times the address associated with a particular Punk has changed.
Please refer to the `punks_sales` DataFrame for the breakdown of all the dates in which a Punk was sold (and the amount it was sold for).

'''
# Getting the amount of times a particular punk has been transferred.
punks_transfer_count = punks_transfer.groupby('punk_id').count().sort_values(by='txn_type', ascending = False)['txn_type'].rename(columns = {'txn_type':'Times Transferred'})
punks_transfer_count.index.rename('Punk ID', inplace = True)

# Top 10 most transferred CryptoPunks:
punks_transfer_count.head(10)

,Times Transferred
Punk ID,
6448,14
9005,13
6276,13
441,12
8389,12
2635,11
7238,11
5287,10
2128,10


In [116]:
# DataFrame with sale date and amount per CryptoPunk.
punks_sales

eth
punk_id date              
0       2018-11-30   25.00
        2017-07-07    1.60
        2017-06-23    0.98
1       2020-11-30   60.00
        2019-04-06   31.00
...                    ...
9997    2021-02-08   99.99
        2020-09-27   59.00
9998    2021-08-11  110.00
        2021-03-03   38.00
        2020-12-27   15.00

[18979 rows x 1 columns]

In [168]:
# DataFrame with transfer date
punks_transfers

Empty DataFrame
Columns: []
Index: [(0, 2017-07-07 00:00:00), (1, 2020-11-30 00:00:00), (1, 2019-06-09 00:00:00), (1, 2019-04-13 00:00:00), (1, 2019-04-06 00:00:00), (1, 2017-07-07 00:00:00), (2, 2021-06-09 00:00:00), (5, 2021-02-04 00:00:00), (5, 2021-02-04 00:00:00), (30, 2021-06-09 00:00:00), (32, 2021-08-13 00:00:00), (32, 2021-02-18 00:00:00), (32, 2021-02-18 00:00:00), (39, 2021-02-04 00:00:00), (39, 2021-02-04 00:00:00), (49, 2021-02-04 00:00:00), (49, 2021-02-04 00:00:00), (51, 2021-02-04 00:00:00), (51, 2021-02-04 00:00:00), (53, 2021-03-17 00:00:00), (58, 2021-06-09 00:00:00), (60, 2021-02-04 00:00:00), (60, 2021-02-04 00:00:00), (63, 2021-02-04 00:00:00), (63, 2021-02-04 00:00:00), (68, 2021-02-04 00:00:00), (68, 2021-02-04 00:00:00), (69, 2021-02-04 00:00:00), (69, 2021-02-04 00:00:00), (74, 2021-02-04 00:00:00), (74, 2021-02-04 00:00:00), (78, 2021-02-04 00:00:00), (78, 2021-02-04 00:00:00), (79, 2021-02-04 00:00:00), (79, 2021-02-04 00:00:00), (84, 2021-02-04 00:00:00), (84, 2021-02-04 00:00:00), (89, 2018-06-12 00:00:00), (95, 2021-02-04 00:00:00), (95, 2021-02-04 00:00:00), (102, 2021-02-05 00:00:00), (103, 2021-03-07 00:00:00), (103, 2021-03-05 00:00:00), (103, 2021-02-04 00:00:00), (108, 2021-03-07 00:00:00), (108, 2021-03-05 00:00:00), (108, 2021-02-04 00:00:00), (109, 2021-03-07 00:00:00), (109, 2021-03-05 00:00:00), (109, 2021-02-04 00:00:00), (111, 2021-05-05 00:00:00), (111, 2021-04-12 00:00:00), (111, 2021-03-07 00:00:00), (111, 2021-03-05 00:00:00), (111, 2021-02-04 00:00:00), (116, 2021-05-15 00:00:00), (118, 2021-03-06 00:00:00), (118, 2021-03-05 00:00:00), (118, 2021-02-04 00:00:00), (119, 2021-02-05 00:00:00), (122, 2021-03-07 00:00:00), (122, 2021-03-05 00:00:00), (122, 2021-02-04 00:00:00), (124, 2021-09-18 00:00:00), (124, 2021-08-23 00:00:00), (124, 2021-03-06 00:00:00), (124, 2021-03-05 00:00:00), (124, 2021-02-04 00:00:00), (127, 2021-03-19 00:00:00), (136, 2021-02-05 00:00:00), (136, 2021-02-04 00:00:00), (136, 2021-02-04 00:00:00), (138, 2021-04-09 00:00:00), (138, 2021-04-06 00:00:00), (138, 2021-03-06 00:00:00), (138, 2021-03-05 00:00:00), (138, 2021-02-04 00:00:00), (139, 2021-03-06 00:00:00), (139, 2021-03-05 00:00:00), (139, 2021-02-04 00:00:00), (141, 2021-04-06 00:00:00), (141, 2021-03-16 00:00:00), (141, 2021-03-07 00:00:00), (141, 2021-03-05 00:00:00), (141, 2021-02-04 00:00:00), (142, 2021-02-05 00:00:00), (144, 2021-02-05 00:00:00), (146, 2021-02-05 00:00:00), (147, 2021-02-05 00:00:00), (149, 2021-02-05 00:00:00), (150, 2021-04-21 00:00:00), (151, 2021-02-05 00:00:00), (152, 2021-02-05 00:00:00), (152, 2021-02-04 00:00:00), (152, 2021-02-04 00:00:00), (153, 2021-02-05 00:00:00), (155, 2021-03-14 00:00:00), (155, 2020-09-11 00:00:00), (156, 2021-02-05 00:00:00), (157, 2021-02-05 00:00:00), ...]

[14225 rows x 0 columns]

In [290]:
url = 'https://api.etherscan.io/api?'
contract_address = '0xb47e3cd837dDF8e4c57F05d70Ab865de6e193BBB'

def get_punk_balance(address, contract_address = contract_address):
    '''
    Get number of punks for a specific address using Etherscan.io API.
    
    Parameters:
    - address
    - contract_address: Defaults to CryptoPunks contract address if none is given.
    
    Returns:
    - Integer with number of ERC20 token balance for address (Number of CryptoPunks that belong to an address, by default).
    
    '''
    request_url = url + f'module=account&action=tokenbalance' \
                        f'&contractaddress={contract_address}' \
                        f'&address={address}' \
                        f'&tag=latest' \
                        f'&apikey={key}' 
    response = requests.get(request_url)
    response = response.json()
    return int(response['result'])

def get_txs_punk(punk_id, start = None, end = None):
    '''
    
    Get all transactions info for a specific punk ID.
    
    Parameters:
    - punk_id: The integer for the CryptoPunk ID.
    - start: String to set a start date for transaction history. Optional- Defaults to None.
    - end: String to set an end date for transaction history. Optional- Defaults to None.
    
    Returns:
    - DataFrame object with columns for transaction type and value in Ether and index for the Punk ID and Date.
    
    '''
    punk_tx = tx_attributes_df[tx_attributes_df['punk_id'] == punk_id]
    punk_tx = punk_tx[['date', 'txn_type', 'eth', 'punk_id']]
    punk_tx.set_index(['punk_id', 'date'], inplace=True)
    if start and end:
        return punk_tx.loc[start:end]
    elif start and not end:
        return punk_tx.loc[start:]
    elif end and not start:
        return punk_tx.loc[:end]
    elif not end and not start:
        return punk_tx

    return tx_attributes_df[tx_attributes_df['punk_id'] == punk_id]

    
def get_txs_address(address, contract_address = contract_address, 
                    start = '0', end = '14561810', offset = '100', sort= 'desc'):
    '''
    
    Usage:
    
    - ERC-20 transfers from an address, specify the address parameter.
    - ERC-20 transfers from a contract address, specify the contract address parameter.
    - ERC-20 transfers from an address filtered by a token contract, specify both address
             and contract address parameters.
    
    Parameters:
    - address: The string representing the address to check for transactions.
    - contract_address: The string representing the token contract address to check for transactions. Defaults to CryptoPunks contract.
    - start: The integer block number to start searching for transactions. Defaults to 0.
    - end: The integer block number to stop searching for transactions. Defaults to 14561810 [Mined on April 11, 2022]
    - sort: The sorting preference, use asc to sort by ascending [oldest at the top] and desc to sort by descending [most recent at the top].
    
    Returns:
    - DataFrame object with 
    
    '''
    request_url = url + f'module=account&action=tokentx' \
                        f'&contractaddress={contract_address}' \
                        f'&address={address}' \
                        f'&page=1' \
                        f'&offset={offset}' \
                        f'&startblock={str(start)}' \
                        f'&endblock={str(end)}' \
                        f'&sort={sort}' \
                        f'&apikey={key}'
    response = requests.get(request_url)
    response = response.json()
    return pd.DataFrame(response['result'])

def get_txn_from_hash(txn_hash, gas_in_wei = True):
    '''
    
    Get information for a transaction through its transaction hash.
    
    Parameters:
    - txn_hash: String for the transaction hash.
    - gas_in_wei: Boolean. If set to `True`, gas prices will be converted to Ether.
    
    Returns:
    - Dictionary with Block Hash, Block Number, Transaction Index, Nonce, Value, & Gas.
    
    '''
    request_url = url + 'module=proxy&action=eth_getTransactionByHash' \
                        f'&txhash={txn_hash}' \
                        f'&apikey={key}'
    response = requests.get(request_url)
    response = response.json()
    data = response['result']
    wei_to_eth = 1000000000000000000
    data['blockNumber'] = int(data['blockNumber'], 16)
    data['transactionIndex'] = int(data['transactionIndex'], 16)
    data['nonce'] = int(data['nonce'], 16)
    data['value'] = round(int(data['value'], 16) / wei_to_eth, 6)
    if gas_in_wei:
        data['gas'] = int(data['gas'], 16)
        data['gasPrice'] = int(data['gasPrice'], 16)
    elif not gas_in_wei:
        data['gas'] = round(int(data['gas'], 16), 6)
        data['gasPrice'] = round(int(data['gasPrice'], 16), 6)
    return data
    